In [ ]:
%pip install  git+https://github.com/CompVis/taming-transformers.git
!git clone https://github.com/CompVis/taming-transformers.git && cd taming-transformers && python -m pip install -e .
%pip install "omegaconf>=2.0.0" "pytorch-lightning>=1.0.8" einops transformers

import os
os._exit(00)

In [ ]:
#VQGAN ImageNet (f=16), 1024

!curl -L "https://heibox.uni-heidelberg.de/d/8088892a516d4e3baf92/files/?p=%2Fckpts%2Flast.ckpt&dl=1" >"last.ckpt"
!curl -L "https://heibox.uni-heidelberg.de/d/8088892a516d4e3baf92/files/?p=%2Fconfigs%2Fmodel.yaml&dl=1" >"model.yaml"


#boris/vqgan_f16_16384
!curl -L "https://huggingface.co/boris/vqgan_f16_16384/raw/main/config.yaml" > "config_vqgan_minidalle.yaml"
!curl -L "https://huggingface.co/boris/vqgan_f16_16384/resolve/main/model.ckpt" > "model_vqgan_minidalle.ckpt"

In [ ]:
import sys
sys.path.append(".")

# also disable grad to save memory
import torch
torch.set_grad_enabled(False)

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

from omegaconf import OmegaConf
import taming
from taming.models.vqgan import VQModel

# config_path = "config_vqgan_minidalle.yaml"
config_path = "model.yaml"

config = OmegaConf.load(config_path)
model=VQModel(**config.model.params)

#ckpt_path = "model_vqgan_minidalle.ckpt"
ckpt_path = "last.ckpt"

sd = torch.load(ckpt_path, map_location="cpu")["state_dict"]
model.load_state_dict(sd, strict=False)

In [ ]:
import torch
import torchvision.transforms.functional as TF
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets.folder import default_loader
from torchvision.transforms import InterpolationMode
import numpy as np
from PIL import Image
def resize_image(image):
      s = min(image.size)
      r = 255 / s
      s = (round(r * image.size[1]), round(r * image.size[0]))
      image = TF.resize(image, s, interpolation=InterpolationMode.LANCZOS)
      image = TF.center_crop(image, output_size = 2 * [255])
      image = np.expand_dims(np.array(image), axis=0)
      return image

img = torch.Tensor(resize_image(Image.open("archive_val/000000287291.jpg"))).permute(0,3, 1, 2)
display(Image.open("archive_val/000000287291.jpg"))
quant, emb_loss, info=model.encode(img)


In [ ]:
logit_laplace_eps: float = 0.1
def unmap_pixels(x: torch.Tensor) -> torch.Tensor:
	if len(x.shape) != 4:
		raise ValueError('expected input to be 4d')
	if x.dtype != torch.float:
		raise ValueError('expected input to have type float')

	return torch.clamp((x - logit_laplace_eps) / (1 - 2 * logit_laplace_eps), 0, 1)

In [ ]:
import torchvision.transforms as T
x_rec=(model.decode(quant)).float()
x_rec=unmap_pixels(torch.sigmoid(x_rec[:, :3]))
T.ToPILImage(mode='RGB')(x_rec[0])